### nn.Module.register_buffer and nn.Parameters
nn.Module.register_buffer特性：
1. 不会产生梯度require_grad = False
2. 不会注册到模型参数中model.parameters()
3. 会注册到模型model.state_dict()中。

In [1]:
import torch

In [ ]:
# 将需要保存到state_dict的变量定义为register_buffer，比如说量化参数。（见quantization.ipynb:QParam）
self.register_buffer('scale', torch.tensor([], requires_grad=False))  
self.register_buffer('zero_point', torch.tensor([], requires_grad=False))
self.register_buffer('min', torch.tensor([], requires_grad=False))
self.register_buffer('max', torch.tensor([], requires_grad=False))

# 对于需要在设备之间转移的变量，必须将其注册为nn.Parameter 或者 buffer，比如说self.M(见quantization.ipynb:QConv2d:freeze)
self.register_buffer('M', torch.tensor([], requires_grad=False))
self.M.data = (self.qw.scale*self.qi.scale / self.qo.scale).data  #赋值tensor而不是对象本身
